In [ ]:
dir = '/content'
vec = dir + '/vectors'
lin = dir + '/links'
vecxz, linxz = vec + '.xz', lin + '.xz'

In [ ]:
%%time
# Priprema datoteke sa vektorima i linkovima
!rm '{vec}' '{lin}' '{vecxz}' '{linxz}'
!gdown --id 1TzbYcSeCPZMLFOEIhMACMSFmjqYm2OB0
!mv 'new_use_questions_and_answers_r_og.xz.xz _ ______' '{vecxz}'

rm: cannot remove '/content/vectors': No such file or directory
rm: cannot remove '/content/links': No such file or directory
rm: cannot remove '/content/vectors.xz': No such file or directory
rm: cannot remove '/content/links.xz': No such file or directory
Downloading...
From: https://drive.google.com/uc?id=1TzbYcSeCPZMLFOEIhMACMSFmjqYm2OB0
To: /content/new_use_questions_and_answers_r_og.xz.xz _ ______
1.42GB [00:11, 128MB/s]
CPU times: user 166 ms, sys: 43 ms, total: 209 ms
Wall time: 32 s


In [ ]:
%%time
# Dekompresovanje datoteke sa vektorima i linkovima
!xz -dk '{vecxz}'

CPU times: user 346 ms, sys: 51.4 ms, total: 398 ms
Wall time: 2min 26s


In [ ]:
%%time
# Ekstrahovanje linkova i formatiranje datoteke sa vektorima
!grep -E -o '^https?.*\n' '{vec}' > '{lin}'
!perl -pi -e 's/https?.*\n//g' '{vec}'
!perl -pi -e 's/(?<=\s)\s+//g' '{vec}'
!perl -pi -e 's/ \((?=-?\d)/\n\(/g' '{vec}'

In [ ]:
# Stampanje metrika formatiranih datoteka
!echo "Total vectors: $( wc -l < {vec} )"
!echo "Total links: $( wc -l < {lin} )"
!echo "Total parentheses: $( grep -E -o '^\(' '{vec}' | wc -l )"

In [ ]:
%%time
# Kompresovanje datoteke sa vektorima i datoteke sa linkovima
!xz -zk -0 '{vec}' '{lin}'

CPU times: user 1.14 s, sys: 157 ms, total: 1.3 s
Wall time: 7min 30s


In [ ]:
#!/bin/bash
declare -i totvec=0
declare -i totlin=0
declare -i totpar=0
for i in {1..5}
do
    path='/mnt/sas1/common-crawl/data/2020-February/ljelic'
    all="$path/new_use_questions_and_answers_r_$i"
    vec="$path/vectors_$i"
    lin="$path/links_$i"
    allxz="$all.xz"
    vecxz="$vec.xz"
    linxz="$lin.xz"
    
    xz -dk $allxz
    
    mv $all $vec

    grep -E -o '^https?.*\n' $vec > $lin
    perl -pi -e 's/https?.*\n//g' $vec
    perl -pi -e 's/(?<=\s)\s+//g' $vec
    perl -pi -e 's/ \((?=-?\d)/\n\(/g' $vec

    cntvec=$((`wc -l < $vec`))
    cntlin=$((`wc -l < $lin`))
    cntpar=$((`grep -E -o '^\(' $vec | wc -l`))

    totvec=$(($totvec + $cntvec))
    totlin=$(($totlin + $cntlin))
    totpar=$(($totpar + $cntpar))

    echo "Total vectors [$i]:\t$cntvec"
    echo "Total links [$i]:\t$cntlin"
    echo "Total parentheses [$i]:\t$cntpar"

    xz -zk -0 $vec $lin
done

echo "Total vectors:\t$totvec"
echo "Total links:\t$totlin"
echo "Total parentheses:\t$totpar"